<a href="https://colab.research.google.com/github/Antique-1/llm-programming1/blob/main/11_04_2_LECL_%EC%B2%B4%EC%9D%B8_%EB%A7%8C%EB%93%A4%EA%B8%B0_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LCEL 체인 만들기
- LCEL 기본 개념 확인
- 파이프(|) 연산자를 사용해서 LangChain의 여러 컴포넌트(모델, 파서, 프롬프트)를 연결하는 체인 구성
- Pydantic을 활용하여 LLM의 출력을 구조화


## 라이브러리 불러오기

In [1]:
%pip install langchain langchain_openai

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.3 MB/s eta 0:00:00


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv

## API KEY 불러오기

In [3]:
api_key = ""
model = ChatOpenAI(
    model="gpt-4o-mini",
    api_key = api_key
    )

## 모델 호출

In [4]:
messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]

model.invoke(messages)

AIMessage(content='안녕하세요, 개스톤. 하지만... 저는 현재 야수와 함께하는 시간을 소중하게 생각하고 있어요. 당신의 제안은 정말 고맙지만, 저녁은 조금 어려울 것 같아요. 다른 분들과의 시간이 더 중요한 것 같아요. 당신은 어떤 일을 하고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 62, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY0hrV6IFaK1po6o3mS47R7GjTJd4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--cb38815b-200c-4f92-b4c7-fede804bdeac-0', usage_metadata={'input_tokens': 62, 'output_tokens': 67, 'total_tokens': 129, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser() #모델의 출력에서 문자열 내용만 추출하는 파서

result = model.invoke(messages)
parser.invoke(result) # AIMessage 객체가 아닌 순수 문자열 출력

'안녕, 개스톤. 너와 저녁을 함께하는 건 고마운 제안이야. 하지만… 나는 다른 생각을 하고 있어. 내 마음은 다른 곳에 있거든. 당신의 매력은 정말 대단하지만, 진짜 사랑은 무엇인지 알고 싶어.'

## 체인 생성

In [6]:
# invoke의 입력이 model로 전달되고, model의 출력이 자동으로 parser로 전달
chain = model | parser
chain.invoke(messages)

'안녕하세요, 개스톤! 당신의 초대는 정말 감사하지만, 저는 당신과 저녁을 먹는 것보다 자연과 책을 좋아해요. 당신의 마음은 아름다워서 많은 사람들에게 사랑받는 것 같아요. 하지만 저는 제 마음이 다른 곳에 있다는 것을 이해해 주실 수 있을까요?'

## 프롬프트 템플릿
- {변수}를 넣어 쉽게 프롬프트를 만들 수 있음
- 딕셔너리 형태로 변수 값을 전달하면 완성된 프롬프트 생성

In [7]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?', additional_kwargs={}, response_metadata={})]


## LCEL 체인 심화

입력 (딕셔너리) -> prompt_template

프롬프트 (메시지) -> model

모델 출력 (AIMessage) -> parser

최종 결과 (문자열)

In [8]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

'안녕하세요, 야수님! 저녁을 함께 한다니 정말 멋진 제안이에요. 당신과 함께 시간을 보낼 수 있어서 기대가 됩니다. 어떤 요리를 좋아하시나요? 함께 정해볼까요?'

## 체인 재사용

In [9]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁"
})

'안녕하세요, 개스톤. 당신의 제안은 정말 고맙지만, 저는 다른 사람과 함께 저녁을 할 계획이 있어요. 당신의 마음은 정말 좋지만, 저에게는 그리움이 있는 사람과 함께하고 싶답니다. 당신도 멋진 저녁을 보내길 바래요.'

## 출력 구조화

### Pydantic 이란?

![이미지](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdna%2FbGANx3%2FbtrUuisjkwf%2FAAAAAAAAAAAAAAAAAAAAANsBUMSIh_Igw32SWHgX13XFytYN8IlVRpkRvRNn0_Vs%2Fimg.webp%3Fcredential%3DyqXZFxpELC7KVnFOS48ylbz2pIh7yKj8%26expires%3D1764514799%26allow_ip%3D%26allow_referer%3D%26signature%3DCu3WpMwN4tLwSv2cVQRNUdrTkcI%253D)

- 원하는 데이터의 스키마(구조)를 파이썬 클래스로 정의하게 해주는 라이브러리
- 런타임 타입 힌트 강제화
- 데이터 직렬화 역직렬화에 사용
- `BaseModel` : Pydantic 라이브러리에서 제공하는 기본 모델 클래스
  - 이 클래스를 상속하여 데이터 모델을 정의하고, 해당 모델의 필드와 유효성 검사 규칙을 설정할 수 있음
- ` Field` : 모델 필드를 정의할 때 사용
  - 필드를 통해 데이터 유효성 검사, 기본값 설정, 제약 조건 등을 세밀하게 제어

In [10]:
from typing import Literal
from pydantic import BaseModel, Field

class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

structured_llm = model.with_structured_output(Adlib) # 모델에게 Adlib Pydantic 클래스의 구조를 반드시 따르도록 설정
adlib_chain = prompt_template | structured_llm

adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
})

Adlib(answer='안녕하세요, 개스톤! 하지만 제 마음은 다른 곳에 있어요. 저는 책을 읽는 것을 좋아하고, 새로운 이야기를 탐험하는 거에요. 당신의 제안은 고맙지만, 저는 독서하는 것이 더 좋을 것 같아요. 혹시 책이나 이야기와 관련된 주제가 있다면 이야기를 나눌 수 있을 것 같네요!', main_emotion='중립', main_emotion_intensity=0.5)

### 문제 1)
BaseModel과 model.with_structured_output()을 함께 사용하는 이유를 적어보세요

In [ ]:
BaseModel로 출력 구조를 설정하고, model.with_structured_output()룰 활용하면 앞서 정의한 구조를 원하는 데이터 형식으로 받을 수 있기 때문에